<a href="https://colab.research.google.com/github/keerthibala07/Machine-learning-models/blob/main/LightGbm_for_spaceship_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
%matplotlib inline
from xgboost import XGBClassifier
from xgboost import plot_tree
from xgboost import plot_importance

train_df = pd.read_csv('/content/titanic train.csv')
test_df  = pd.read_csv('/content/titanic test.csv')


In [ ]:
train_df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [ ]:
train_df.loc[train_df['FoodCourt'] > 20000, 'FoodCourt'] = train_df.loc[train_df['FoodCourt'] < 20000, 'FoodCourt'].mean()

In [ ]:
train_df.loc[train_df['ShoppingMall'] > 10000, 'ShoppingMall'] = train_df.loc[train_df['ShoppingMall'] < 10000, 'ShoppingMall'].mean()

In [ ]:
train_df.loc[train_df['Spa'] > 20000, 'Spa'] = train_df.loc[train_df['Spa'] < 20000, 'Spa'].mean()

In [ ]:
train_df.loc[train_df['VRDeck'] > 20000, 'VRDeck'] = train_df.loc[train_df['VRDeck'] < 20000, 'VRDeck'].mean()

In [ ]:
train_df.loc[train_df['RoomService'] > 10000, 'RoomService'] = train_df.loc[train_df['RoomService'] < 10000, 'RoomService'].mean()

In [ ]:
train_df['Age'].fillna(train_df['Age'].mean())
train_df.fillna(method='ffill',inplace=True)
test_df['Age'].fillna(test_df['Age'].mean())
test_df.fillna(method='ffill',inplace=True)
train_df[['DeckName','DeckNumber','DeckSide']]=train_df['Cabin'].str.split('/',expand=True)
test_df[['DeckName','DeckNumber','DeckSide']]=train_df['Cabin'].str.split('/',expand=True)
le = preprocessing.LabelEncoder()
def converting_column(df,col,convertType):
    if convertType=='OHE':
        one_hot = pd.get_dummies(df[col],prefix=col)
        df = df.drop(col,axis = 1)
        df = df.join(one_hot)
        return df
    else:
        df[col] = le.fit_transform(df[col])
        return df

train_df = converting_column(train_df,'HomePlanet','OHE')
test_df = converting_column(test_df,'HomePlanet','OHE')
train_df = converting_column(train_df,'Destination','FTR')
test_df = converting_column(test_df,'Destination','FTR')
train_df = converting_column(train_df,'CryoSleep','FTR')
test_df = converting_column(test_df,'CryoSleep','FTR')
train_df = converting_column(train_df,'VIP','FTR')
test_df = converting_column(test_df,'VIP','FTR')
train_df['DeckName'] = le.fit_transform(train_df['DeckName'])
train_df['DeckSide'] = le.fit_transform(train_df['DeckSide'])
test_df['DeckName'] = le.fit_transform(test_df['DeckName'])
test_df['DeckSide'] = le.fit_transform(test_df['DeckSide'])
train_df.drop(['PassengerId','Cabin','HomePlanet_Earth','HomePlanet_Europa','HomePlanet_Mars','DeckNumber','Name'],axis=1,inplace=True)
test_df.drop(['PassengerId','Cabin','HomePlanet_Earth','HomePlanet_Europa','HomePlanet_Mars','DeckNumber','Name'],axis=1,inplace=True)

In [ ]:
train_df.corr()

,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,DeckName,DeckSide
CryoSleep,1.000000,-0.095649,-0.068900,-0.074514,-0.245922,-0.211081,-0.236741,-0.196275,-0.195068,0.456062,0.014606,0.017731
Destination,-0.095649,1.000000,-0.009812,-0.043064,0.046501,-0.104069,0.019249,-0.055782,-0.071393,-0.108732,0.182895,-0.010304
Age,-0.068900,-0.009812,1.000000,0.090322,0.068285,0.132955,0.042965,0.118026,0.097718,-0.074233,-0.241791,0.009412
VIP,-0.074514,-0.043064,0.090322,1.000000,0.055266,0.107112,0.022379,0.059519,0.126146,-0.034560,-0.177189,-0.008651
RoomService,-0.245922,0.046501,0.068285,0.055266,1.000000,-0.016580,0.068514,0.011897,-0.018323,-0.245273,-0.026712,-0.009748
FoodCourt,-0.211081,-0.104069,0.132955,0.107112,-0.016580,1.000000,-0.013650,0.215470,0.233889,0.039947,-0.320673,0.017244
ShoppingMall,-0.236741,0.019249,0.042965,0.022379,0.068514,-0.013650,1.000000,-0.004335,-0.021356,0.008826,-0.013254,-0.022445
Spa,-0.196275,-0.055782,0.118026,0.059519,0.011897,0.215470,-0.004335,1.000000,0.156014,-0.219774,-0.224039,0.009333
VRDeck,-0.195068,-0.071393,0.097718,0.126146,-0.018323,0.233889,-0.021356,0.156014,1.000000,-0.209415,-0.252568,-0.008848
Transported,0.456062,-0.108732,-0.074233,-0.034560,-0.245273,0.039947,0.008826,-0.219774,-0.209415,1.000000,-0.114933,0.103142


In [ ]:
train_df['kids'] = train_df['Age'].apply(lambda x: 1 if x < 15 else 0)
test_df['kids'] = test_df['Age'].apply(lambda x: 1 if x < 15 else 0)
train_df = train_df.drop(['Age'], axis=1)
test_df = test_df.drop(['Age'], axis=1)

In [ ]:
x_train= train_df.drop("Transported", axis=1)
y_train= train_df["Transported"]

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size = 0.25, random_state =1)

In [ ]:
params = {'learning_rate': 0.13,
          'min_child_weight': 0.15,
          'gamma': 0.25,
          'subsample': 0.65,
          'colsample_bytree': 0.85,
          'colsample_bylevel': 0.64,
          'max_depth': 4,
          'n_estimators': 4500,
          'reg_lambda': 2.25,
          'monotone_constraints': '(0, -1)',
          'use_label_encoder': False
          }
xgb_model = XGBClassifier()
xgb_model.set_params(**params)
mod = xgb_model.fit(X_train, Y_train)

print("Performance on train data:", mod.score(X_train, Y_train))
print(mod.score(X_test,Y_test))

Performance on train data: 0.9148642429820525
0.7953081876724931


In [ ]:
x= train_df.drop("Transported", axis=1)
y= train_df["Transported"]

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(x, y, test_size = 0.20, random_state =1)

import lightgbm as lgb
clf=lgb.LGBMClassifier(learning_rate= 0.1, max_depth= 7,num_leaves=25,n_estimators=2000,colsample_bytree=0.85, colsample_bylevel = 0.64 )
clf.fit(x_train,y_train)

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001872 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1294
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

LGBMClassifier(colsample_bylevel=0.64, colsample_bytree=0.85, max_depth=7,
               n_estimators=2000, num_leaves=25)

In [ ]:
y_pred=clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(Y_test,y_pred)
print(accuracy*100)

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
92.35192639447959


In [ ]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Assuming x_train, y_train are your features and target variable
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20, random_state=1)

# Create LGBMClassifier
clf = lgb.LGBMClassifier()

# Define the hyperparameter grid
param_grid = {
    'learning_rate': [0.06, 0.1, 0.5],
    'max_depth': [5, 7, 10],
    'num_leaves': [20, 25, 30],
    'n_estimators': [100, 500, 1000],
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)

# Fit the model to the training data
grid_search.fit(X_train, Y_train)

# Print the best parameters and corresponding accuracy
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)

# Make predictions on the test set using the best model
Y_pred = grid_search.best_estimator_.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, Y_pred)
print("Test Accuracy: ", accuracy)


Fitting 3 folds for each of 81 candidates, totalling 243 fits
[LightGBM] [Info] Number of positive: 3482, number of negative: 3472
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001451 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1294
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500719 -> initscore=0.002876
[LightGBM] [Info] Start training from score 0.002876
Best Parameters:  {'learning_rate': 0.06, 'max_depth': 10, 'n_estimators': 100, 'num_leaves': 30}
Best Accuracy:  0.7942191544434859
Test Accuracy:  0.8016101207590569
